# MLRun send email
MLRun function to send emails using SMTP

## MLRun conf

In [1]:
%load_ext lab_black

from mlrun import mlconf
import os

# artifact_path = mlconf.artifact_path or os.path.abspath('jobs')
artifact_path = os.path.abspath("jobs")
mlconf.dbpath = "http://mlrun-api:8080"
mlconf.artifact_path = artifact_path
print(f"Artifacts path: {mlconf.artifact_path}\nMLRun DB path: {mlconf.dbpath}")

Artifacts path: /User/functions/send_email/jobs
MLRun DB path: http://mlrun-api:8080


## Save function

In [2]:
import yaml

with open("item.yaml") as item_file:
    items = yaml.load(item_file, Loader=yaml.FullLoader)

In [3]:
from mlrun import code_to_function

# create job function object from notebook code
fn = code_to_function(
    name=items["name"],
    kind=items["spec"]["kind"],
    handler=items["spec"]["handler"],
    filename=items["spec"]["filename"],
    image=items["spec"]["image"],
    description=items["description"],
    categories=items["categories"],
    labels=items["labels"],
)

fn.export("send_email.yaml")

> 2021-02-17 11:00:13,857 [info] function spec saved to path: send_email.yaml


# Examples
First, configure MLRun. Define project parameters that will be used for testing the function

In [4]:
from os import path, getenv
from mlrun import new_project

project_name = "-".join(filter(None, ["email-sending", getenv("V3IO_USERNAME", None)]))
project_path = path.abspath("conf")
project = new_project(project_name, project_path, init_git=True)

print(f"Project path: {project_path}\nProject name: {project_name}")

Project path: /User/functions/send_email/conf
Project name: email-sending-admin


In [5]:
from mlrun import run_local, NewTask, import_function, mount_v3io

# Target location for storing pipeline artifacts
artifact_path = path.abspath("jobs")

# MLRun DB path or API service URL
mlconf.dbpath = mlconf.dbpath or "http://mlrun-api:8080"

print(f"Artifacts path: {artifact_path}\nMLRun DB path: {mlconf.dbpath}")

Artifacts path: /User/functions/send_email/jobs
MLRun DB path: http://mlrun-api:8080


## Create some artifacts

First we'll load the Iris dataset and use the describe function to generate some artifacts describing it. 
>This is only used to generate some nice artifacts so we can send them later via email. If all you want is to test the email sending functionality you can safely ignore this part (and modify the code that actually sends the email to not use attachments).


In [6]:
from send_email import send_email
from mlrun.execution import MLClientCtx
from typing import List
from os import path
import pandas as pd

# Ingest a data set into the platform
def get_data(context, source_url, format="csv"):

    iris_dataset = pd.read_csv(str(source_url))

    target_path = path.join(context.artifact_path, "data")
    # Optionally print data to your logger
    context.logger.info("Saving Iris data set to {} ...".format(target_path))

    # Store the data set in your artifacts database
    context.log_dataset(
        "iris_dataset",
        df=iris_dataset,
        format=format,
        index=False,
        artifact_path=target_path,
    )


source_url = "http://iguazio-sample-data.s3.amazonaws.com/iris_dataset.csv"

get_data_run = run_local(
    name="get_data",
    handler=get_data,
    inputs={"source_url": source_url},
    project=project_name,
    artifact_path=artifact_path,
)

project.set_function("hub://describe", "describe")
describe = project.func("describe").apply(mount_v3io())

describe_run = describe.run(
    params={"label_column": "label"},
    inputs={"table": get_data_run.outputs["iris_dataset"]},
    artifact_path=artifact_path,
)

> 2021-02-17 11:00:14,142 [info] starting run get_data uid=2f33111e836141338512b06a8ed15c37 DB=http://mlrun-api:8080
> 2021-02-17 11:00:14,767 [info] Saving Iris data set to /User/functions/send_email/jobs/data ...


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
email-sending-admin,...8ed15c37,0,Feb 17 11:00:14,completed,get_data,v3io_user=adminkind=handlerowner=adminhost=jupyter-7b854d9bd6-mkmbn,source_url,,,iris_dataset


to track results use .show() or .logs() or in CLI: 
!mlrun get run 2f33111e836141338512b06a8ed15c37 --project email-sending-admin , !mlrun logs 2f33111e836141338512b06a8ed15c37 --project email-sending-admin
> 2021-02-17 11:00:15,035 [info] run executed, status=completed
> 2021-02-17 11:00:15,623 [info] starting run describe-summarize uid=fcc766f980564ce1b238548cc00a5a61 DB=http://mlrun-api:8080
> 2021-02-17 11:00:16,021 [info] Job is running in the background, pod: describe-summarize-2s2v8
> 2021-02-17 11:00:27,416 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
email-sending-admin,...c00a5a61,0,Feb 17 11:00:21,completed,describe-summarize,v3io_user=adminkind=jobowner=adminhost=describe-summarize-2s2v8,table,label_column=label,,histogramsviolinimbalanceimbalance-weights-veccorrelation-matrixcorrelation


to track results use .show() or .logs() or in CLI: 
!mlrun get run fcc766f980564ce1b238548cc00a5a61 --project email-sending-admin , !mlrun logs fcc766f980564ce1b238548cc00a5a61 --project email-sending-admin
> 2021-02-17 11:00:35,403 [info] run executed, status=completed


## Sending the email

Sending emails need to have server_addr set to confirure the SMTP address. It also needs to have secrets created with the SMTP_USER and SMTP_PASSWORD secrets set, so it can login to the server.

We'll send an email with the artifacts generated by the describe function. Note that some of these artifacts are HTML and the last one is a CSV. The send_email function will attempt to auto-detect the attachments' MIME types and add them to the email with their appropriate types.

## Configure task parameters to be used when executing the function

***Make sure to replace placeholders with actual SMTP configuration (address/email/password)***

In [7]:
task_params = {
    "sender": "<sender email>",
    "to": "<recipient email>",
    "subject": "Dataset description, sent by the send_email function",
    "content": "Some basic analysis of the iris dataset.",
    "attachments": [
        describe_run.outputs["histograms"],
        describe_run.outputs["correlation"],
        describe_run.outputs["correlation-matrix"],
    ],
    "server_addr": "<server address>:<port>",
}

task = NewTask(
    name="email_task",
    project=project_name,
    handler=send_email,
    artifact_path=artifact_path,
    params=task_params,
)

task_secrets = {"SMTP_USER": "<username>", "SMTP_PASSWORD": "<password>"}

task.with_secrets("inline", task_secrets)

## Run locally

In [8]:
send_email_run = run_local(task,name='send_email')

> 2021-02-17 11:00:35,466 [info] starting run send_email uid=3e7d60dc59ef4bbb9f11a6d61680cee9 DB=http://mlrun-api:8080
> 2021-02-17 11:00:35,699 [info] Looking at attachment: /User/functions/send_email/jobs/plots/hist.html
> 2021-02-17 11:00:35,713 [info] Added attachment: Filename: /User/functions/send_email/jobs/plots/hist.html, of mimetype: text, html
> 2021-02-17 11:00:35,713 [info] Looking at attachment: /User/functions/send_email/jobs/plots/corr.html
> 2021-02-17 11:00:35,718 [info] Added attachment: Filename: /User/functions/send_email/jobs/plots/corr.html, of mimetype: text, html
> 2021-02-17 11:00:35,719 [info] Looking at attachment: /User/functions/send_email/jobs/plots/correlation-matrix.csv
> 2021-02-17 11:00:35,722 [info] Added attachment: Filename: /User/functions/send_email/jobs/plots/correlation-matrix.csv, of mimetype: text, csv
> 2021-02-17 11:00:35,736 [error] Traceback (most recent call last):
  File "/conda/lib/python3.7/smtplib.py", line 329, in connect
    port =

nonnumeric port


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
email-sending-admin,...1680cee9,0,Feb 17 11:00:35,error,send_email,v3io_user=adminkind=handlerowner=adminhost=jupyter-7b854d9bd6-mkmbn,,"sender=to=subject=Dataset description, sent by the send_email functioncontent=Some basic analysis of the iris dataset.attachments=['/User/functions/send_email/jobs/plots/hist.html', '/User/functions/send_email/jobs/plots/corr.html', '/User/functions/send_email/jobs/plots/correlation-matrix.csv']server_addr=:",,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 3e7d60dc59ef4bbb9f11a6d61680cee9 --project email-sending-admin , !mlrun logs 3e7d60dc59ef4bbb9f11a6d61680cee9 --project email-sending-admin
> 2021-02-17 11:00:36,033 [info] run executed, status=error


RunError: nonnumeric port

## Run remotely

In [11]:
%load_ext lab_black

# Convert the local get_data function into an email_func project function
email_func = code_to_function(
    name=items["name"],
    kind=items["spec"]["kind"],
    handler=items["spec"]["handler"],
    filename=items["spec"]["filename"],
    image=items["spec"]["image"],
    description=items["description"],
    categories=items["categories"],
    labels=items["labels"],
)
email_func.apply(mount_v3io())

email_func.run(task, params=task_params,  workdir=mlconf.artifact_path)
email_func.doc()

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
